# 1. Import de Bibliotecas e de Algoritmos

In [10]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import json


# Importar modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

# 2. FEATURE ENGINEERING - Extracao e Limpeza de Dados 

In [11]:
def feature_engeneering(df):
    df_eng = df.copy()

    # --- LIMPEZA INICIAL ---
    df_eng = df_eng.drop_duplicates()
    df_eng['hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False).astype(float)
    df_eng['liters'] = df['engine'].str.extract(r'(\d+\.?\d*)L\s', expand=False).astype(float)

    # --- Idade e Uso ---
    var_ano_atual = date.today().year
    df_eng['car_age'] = var_ano_atual - df_eng['model_year']
    df_eng['car_age'] = df_eng['car_age'].replace(0, 1)

    # --- Cilindrada ---
    df_eng['cylinders'] = df['engine'].str.extract(r'(\d+)\s+Cylinder', expand=False)
    df_eng['cylinders'] = df_eng['cylinders'].fillna(df['engine'].str.extract(r'V(\d+)', expand=False))
    df_eng['cylinders'] = df_eng['cylinders'].astype(float)

    # --- Tecnologias de Motor ---
    df_eng['is_turbo'] = df['engine'].str.contains(r'(?i)turbo', na=False).astype(int)
    df_eng['turbo_type'] = df['engine'].str.extract(r'(Twin Turbo|Turbo)', expand=False)
    df_eng['valve_train'] = df['engine'].str.extract(r'(DOHC|OHV|SOHC)', expand=False) 
    df_eng['fuel_injection'] = df['engine'].str.extract(r'(PDI|GDI|MPFI)', expand=False)

    # Miles per year
    df_eng['miles_p_year'] = df_eng['milage'] / df_eng['car_age']

    # --- FUEL TYPE ---
    def clean_fuel(val):
        s = str(val).lower()
        if 'hybrid' in s:
            return 'Hybrid'
        elif 'not supported' in s:
            return 'EV'
        else:
            return val
    df_eng['fuel_type'] = df_eng['fuel_type'].apply(clean_fuel)

    # --- TRANSMISSION TYPE ---
    def clean_transmission(val):
        s = str(val).lower()
        if 'automatic' in s or 'a/t' in s or 'cvt' in s:
            return 'Automatico'
        elif 'manual' in s or 'm/t' in s:
            return 'Manual'
        else:
            return 'Outro'
    df_eng['transmission_type'] = df_eng['transmission'].apply(clean_transmission)

    # --- Cores ---
    top_ext_colors = df_eng['ext_col'].value_counts().nlargest(10).index
    def simplificar_cor_ext(cor):
        return cor if cor in top_ext_colors else 'Other'
    df_eng['ext_col_simple'] = df_eng['ext_col'].apply(simplificar_cor_ext)

    top_int_colors = df_eng['int_col'].value_counts().nlargest(10).index
    def simplificar_cor_int(cor):
        return cor if cor in top_int_colors else 'Other'
    df_eng['int_col_simple'] = df_eng['int_col'].apply(simplificar_cor_int)

    # --- Tratamento de Nulos ---
    cols_texto = df_eng.select_dtypes(include=['object']).columns
    df_eng[cols_texto] = df_eng[cols_texto].replace('-', 'Unknown').fillna('Unknown')
    df_eng['clean_title'] = df_eng['clean_title'].replace('Unknown', 'No')

    # --- Acidente ---
    def verificar_acidente(valor):
        return 0 if 'None' in str(valor) else 1
    df_eng['accident_clean'] = df_eng['accident'].apply(verificar_acidente)

    # 1. Rácio de Potência por Litro (Eficiência do motor)
    # Evitar divisão por zero somando um valor ínfimo
    df_eng['hp_per_liter'] = df_eng['hp'] / (df_eng['liters'] + 0.001)

    # 2. Rácio de Potência por Cilindro
    df_eng['hp_per_cylinder'] = df_eng['hp'] / (df_eng['cylinders'] + 0.001)

    # 3. Log na Quilometragem (Milage)
    # A milage tem uma distribuição muito "cauda longa". O Log ajuda o modelo a ver melhor as diferenças.
    df_eng['milage_log'] = np.log1p(df_eng['milage'])

    return df_eng

# 3. PREPARAÇÃO DE DADOS

In [12]:
def preparar_dados(df_treino, df_teste):
    """Prepara dados para modelagem"""

    # Aplicar feature engineering
    df_treino_eng = feature_engeneering(df_treino)
    df_teste_eng = feature_engeneering(df_teste)

    # Separar target
    y = df_treino_eng['price']
    X = df_treino_eng.drop('price', axis=1)
    X_test = df_teste_eng.copy()

    # Selecionar features relevantes
    features_numericas = ['hp', 'liters', 'car_age', 'cylinders', 'miles_p_year', 
                          'milage', 'model_year', 'is_turbo']

    features_categoricas = ['brand', 'model', 'fuel_type', 'transmission_type', 
                           'ext_col_simple', 'int_col_simple', 'clean_title', 
                           'turbo_type', 'valve_train', 'fuel_injection']

    # Criar dataset numérico
    X_num = X[features_numericas].fillna(0)
    X_test_num = X_test[features_numericas].fillna(0)

    # Encodar variáveis categóricas
    X_cat = X[features_categoricas].copy()
    X_test_cat = X_test[features_categoricas].copy()

    encoders = {}
    for col in features_categoricas:
        le = LabelEncoder()
        # Fit no treino
        X_cat[col] = X_cat[col].astype(str)
        le.fit(X_cat[col])
        X_cat[col] = le.transform(X_cat[col])

        # Transform no teste (tratando categorias novas)
        X_test_cat[col] = X_test_cat[col].astype(str)
        X_test_cat[col] = X_test_cat[col].apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1
        )
        encoders[col] = le

    # Concatenar features
    X_final = pd.concat([X_num, X_cat], axis=1)
    X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

    return X_final, y, X_test_final, encoders

# 4. DEFINIÇÃO DE MODELOS

In [13]:
def obter_modelos():
    """Retorna dicionário com todos os modelos disponíveis"""

    modelos = {
        # Regressão Linear
        'Linear Regression': LinearRegression(),

        # K-Nearest Neighbors
        #'KNN': KNeighborsRegressor(),

        # Árvores de Decisão
        'Decision Tree': DecisionTreeRegressor(random_state=42),

        # XGBoost
        'XGBoost': xgb.XGBRegressor(random_state=42, n_jobs=-1),

        # Support Vector Machines
        'SVR Linear': SVR(kernel='linear'),
        #'SVR RBF': SVR(kernel='rbf'),

        # Redes Neuronais
        'MLP Small': MLPRegressor(random_state=42, early_stopping=True),
        #'MLP Deep': MLPRegressor(random_state=42, early_stopping=True)
    }

    return modelos

# 5. GridSearchCV - Grid Search com Cross Validation

In [14]:
# 5. GridSearchCV - Otimização de Hiperparâmetros
def obter_params_grid(nome_modelo):
    """
    Retorna o dicionário de hiperparâmetros para testar baseado no nome do modelo.
    """
    grids = {
        'Linear Regression': {
            'fit_intercept': [True, False],
            'positive': [False, True]
        },
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'XGBoost': {
            'n_estimators': [100, 500, 1000],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'subsample': [0.7, 0.9, 1.0],
            'colsample_bytree': [0.7, 0.9, 1.0],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.5, 1.0]
        },
        'KNN': {
            'n_neighbors': [3, 5, 7, 11],
            'weights': ['uniform', 'distance'],
            'p': [1, 2]
        },
        'Decision Tree': {
            'max_depth': [5, 10, 15, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'SVR Linear': {
            'C': [0.1, 1, 10, 100],
            'epsilon': [0.01, 0.1, 0.5]
        },
        'SVR RBF': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.1, 0.01],
            'epsilon': [0.01, 0.1, 0.5]
        },
        'MLP Small': {
            'hidden_layer_sizes': [(50,), (100,), (50, 25)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001]
        },
        'MLP Deep': {
            'hidden_layer_sizes': [(100, 50), (100, 50, 25), (50, 50, 50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001]
        }
    }
    return grids.get(nome_modelo, {})

def executar_grid_search(modelo, nome_modelo, X, y, usar_scaled=False):
    """
    Executa o GridSearch para encontrar os melhores hiperparâmetros.
    """
    print(f"\n🔍 Iniciando GridSearch para: {nome_modelo}")

    # 1. Obter a grelha de parâmetros
    param_grid = obter_params_grid(nome_modelo)

    if not param_grid:
        print(f"⚠️ Nenhuma grid definida para {nome_modelo}. Retornando modelo base.")
        return modelo

    # 2. Configurar o GridSearch
    grid_search = GridSearchCV(
        estimator=modelo,
        param_grid=param_grid,
        cv=3, 
        scoring='neg_root_mean_squared_error',
        verbose=1,
        n_jobs=-1  # Usa todos os processadores
    )

    # 3. Treinar
    grid_search.fit(X, y)

    # 4. Resultados
    print(f"✅ Melhores Parâmetros: {grid_search.best_params_}")
    print(f"   Melhor RMSE (CV): {-grid_search.best_score_:,.2f}")
    return grid_search.best_estimator_

# 6. AVALIAÇÃO DE MODELOS

In [15]:
def avaliar_modelo(modelo, X_train, y_train, X_val, y_val, nome_modelo):
    """Treina e avalia um modelo"""

    print(f"\n{'='*60}")
    print(f"Treinando: {nome_modelo}")
    print(f"{'='*60}")

    # Treinar
    modelo.fit(X_train, y_train)

    # Predições
    y_train_pred = modelo.predict(X_train)
    y_val_pred = modelo.predict(X_val)

    # Métricas de treino
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

    # Métricas de validação
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

    # Exibir resultados
    print(f"\n📊 MÉTRICAS DE TREINO:")
    print(f"  RMSE: {train_rmse:,.2f}")

    print(f"\n📊 MÉTRICAS DE VALIDAÇÃO:")
    print(f"  RMSE: {val_rmse:,.2f}")

    return {
        'modelo': nome_modelo,
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'modelo_treinado': modelo
    }

# 7. TREINAR MODELOS

In [16]:
def treinar_modelos(df_treino, df_teste, modelos_selecionados=None):
    """
    Treina e compara múltiplos modelos

    Args:
        df_treino: DataFrame de treino
        df_teste: DataFrame de teste
        modelos_selecionados: Lista com nomes dos modelos a treinar (None = todos)
    """

    print("🔄 Preparando dados...")
    X, y, X_test, encoders = preparar_dados(df_treino, df_teste)

    # Aplicar log-transform ao target (importante para distribuição dos preços)
    y_log = np.log1p(y)

    # Split treino/validação
    X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

    # Normalizar dados (importante para KNN, SVM e MLP)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)



    print(f"✅ Dados preparados:")
    print(f"  - Treino: {X_train.shape[0]} amostras")
    print(f"  - Validação: {X_val.shape[0]} amostras")
    print(f"  - Features: {X_train.shape[1]}")

    # Obter modelos
    todos_modelos = obter_modelos()

    # Filtrar modelos se especificado
    if modelos_selecionados:
        modelos = {k: v for k, v in todos_modelos.items() if k in modelos_selecionados}
    else:
        modelos = todos_modelos

    # Treinar modelos
    resultados = []
    modelos_treinados = {}

    for nome, modelo in modelos.items():
        # Decidir se usar dados normalizados
        usar_scaled = nome in ['KNN', 'SVR Linear', 'SVR RBF', 'MLP Small', 'MLP Deep']

        if usar_scaled:
            # Executar GridSearch para otimizar hiperparâmetros
            modelo_otimizado = executar_grid_search(modelo, nome, X_train_scaled, y_train, usar_scaled=True)
            res = avaliar_modelo(modelo_otimizado, X_train_scaled, y_train, 
                                X_val_scaled, y_val, nome)
        else:
            # Executar GridSearch para otimizar hiperparâmetros
            modelo_otimizado = executar_grid_search(modelo, nome, X_train, y_train, usar_scaled=False)
            res = avaliar_modelo(modelo_otimizado, X_train, y_train, 
                                X_val, y_val, nome)

        resultados.append(res)
        modelos_treinados[nome] = {
            'modelo': res['modelo_treinado'],
            'usar_scaled': usar_scaled
        }

    # Comparação final
    print(f"\n\n{'='*80}")
    print("📈 COMPARAÇÃO DE MODELOS (ordenados por RMSE de validação)")
    print(f"{'='*80}\n")

    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values('val_rmse', ascending=True)

    print(df_resultados[['modelo', 'train_rmse', 'val_rmse']].to_string(index=False))

    # Melhor modelo
    melhor = df_resultados.iloc[0]
    print(f"\n\n🏆 MELHOR MODELO: {melhor['modelo']}")
    print(f"  RMSE Validação: ${melhor['val_rmse']:,.2f}")

    return df_resultados, modelos_treinados, scaler, (X_test, X_test_scaled)

# 8. Guardar Hiperparametros 

In [17]:
def salvar_submissao_log(df_sub, modelo_treinado, nome_modelo, metricas):
    """
    Salva CSV e JSON incrementando o ID com base no maior número encontrado.
    """
    pasta = 'submissoes'
    os.makedirs(pasta, exist_ok=True)

    # 1. Listar arquivos e encontrar o maior ID existente
    arquivos = os.listdir(pasta)
    ids_existentes = []

    for f in arquivos:
        # Verifica se o arquivo segue o padrão 'submission_X.csv'
        if f.startswith('submission_') and f.endswith('.csv'):
            try:
                # Extrai apenas o número do nome do arquivo
                # Ex: 'submission_12.csv' -> '12'
                numero_str = f.replace('submission_', '').replace('.csv', '')
                ids_existentes.append(int(numero_str))
            except ValueError:
                continue # Pula arquivos que não tenham número válido

    # Se a lista estiver vazia, começa do 1. Se não, pega o maior + 1
    if not ids_existentes:
        next_id = 1
    else:
        next_id = max(ids_existentes) + 1

    # 2. Definir nomes dos arquivos
    filename_csv = f"{pasta}/submission_{next_id}.csv"
    filename_json = f"{pasta}/submission_{next_id}_params.json"

    # 3. Salvar CSV
    df_sub.to_csv(filename_csv, index=False)

    # 4. Extrair Hiperparâmetros
    try:
        params = modelo_treinado.get_params()
    except:
        params = {"info": "Não foi possível extrair params"}

    # 5. Metadata
    metadata = {
        "id": next_id,
        "modelo": nome_modelo,
        "performance_validacao": metricas,
        "hiperparametros": params
    }

    # 6. Salvar JSON
    with open(filename_json, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=4, default=str)

    print(f"\n✅ Submissão #{next_id} salva com sucesso!")
    print(f"   📂 {filename_csv}")

# Fim - Execucao do Main

In [ ]:
if __name__ == "__main__":
    # 1. Carregar dados
    print("📂 Carregando dados...")
    df_treino = pd.read_csv('dados/train.csv', index_col='id')
    df_teste = pd.read_csv('dados/test.csv', index_col='id')

    # 2. Treinar e comparar múltiplos modelos
    print("\n🚀 INICIANDO TREINO DE MÚLTIPLOS MODELOS...\n")
    df_resultados, modelos_treinados, scaler, (X_test, X_test_scaled) = treinar_modelos(df_treino, df_teste)

    # 3. Selecionar melhor modelo
    melhor_resultado = df_resultados.iloc[0]
    melhor_modelo_nome = melhor_resultado['modelo']
    melhor_modelo = modelos_treinados[melhor_modelo_nome]['modelo']
    usar_scaled = modelos_treinados[melhor_modelo_nome]['usar_scaled']

    # 4. Fazer previsões no teste
    if usar_scaled:
        X_test_usar = X_test_scaled
    else:
        X_test_usar = X_test

    pred_log = melhor_modelo.predict(X_test_usar)
    pred_reais = np.expm1(pred_log)

    # 5. Submissão
    print(f"\n🏆 Gerando submissão com melhor modelo: {melhor_modelo_nome}...")
    df_submissao = pd.DataFrame({
        'id': df_teste.index,
        'price': pred_reais
    })

    salvar_submissao_log(df_submissao, melhor_modelo, melhor_modelo_nome, melhor_resultado.to_dict())

📂 Carregando dados...

🚀 INICIANDO TREINO DE MÚLTIPLOS MODELOS...

🔄 Preparando dados...
✅ Dados preparados:
  - Treino: 150826 amostras
  - Validação: 37707 amostras
  - Features: 18

🔍 Iniciando GridSearch para: Linear Regression
Fitting 3 folds for each of 4 candidates, totalling 12 fits
✅ Melhores Parâmetros: {'fit_intercept': True, 'positive': False}
   Melhor RMSE (CV): 0.54

Treinando: Linear Regression

📊 MÉTRICAS DE TREINO:
  RMSE: 0.54

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: 0.54

🔍 Iniciando GridSearch para: Decision Tree
Fitting 3 folds for each of 36 candidates, totalling 108 fits
✅ Melhores Parâmetros: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
   Melhor RMSE (CV): 0.52

Treinando: Decision Tree

📊 MÉTRICAS DE TREINO:
  RMSE: 0.49

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: 0.51

🔍 Iniciando GridSearch para: XGBoost
Fitting 3 folds for each of 2916 candidates, totalling 8748 fits
✅ Melhores Parâmetros: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'm